In [ ]:
#1st step: Let's import my Google Drive trajectory here so we can easily load the needed files.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#2nd step: We need to install COBRA if it's not already installed on Google Colab.
!pip install cobra

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 49.5 MB/s eta 0:00:00


In [ ]:
#3rd step: Let's import the two libraries needed for our task. We require COBRA to load the model and categorize the genes as either essential or non-essential. Additionally, we'll import CSV to export the results and create a spreadsheet from them.
import cobra
import csv

In [ ]:
#4th step: Here, we name our model and read the SBML file from my Google Drive. Since this file has already been uploaded to Drive once, importing it here means we don't need to upload it again in Colab.
model = cobra.io.read_sbml_model('/content/drive/MyDrive/YL-MODELS/MODEL1111190000.xml')

Streaming output truncated to the last 5000 lines.


In [ ]:
#5th step: Let's perform Flux Balance Analysis (FBA). At this stage, we haven't added or removed any nutrients from the medium; our goal is to identify the essential genes in the current model.
solution = model.optimize()

In [ ]:
#6th step: First, we define the lists 'essential_genes' and 'non_essential_genes'.
essential_genes = []
non_essential_genes = []
#Finally, we iterate through the genes by creating a 'for loop' and perform knockout simulations to distinguish the essential genes from the non-essential ones.
# If the objective value after knocking out the gene is less than 1% of the original objective value, it indicates that knocking out the gene significantly impairs the growth rate compared to the original condition.
for gene in model.genes:
    with model:
        gene.knock_out()
        knockout_solution = model.optimize()
        if knockout_solution.status == 'infeasible' or knockout_solution.objective_value < 0.01 * solution.objective_value:
            essential_genes.append(gene.id)
        elif knockout_solution.status == 'optimal':
            non_essential_genes.append(gene.id)

In [1]:
#Now, let's verify whether the function has been completed by checking if the essential and non-essential genes have been properly categorized.
print("Essential Genes:", essential_genes)

SyntaxError: unterminated string literal (detected at line 1) (<ipython-input-1-eff2acaa0034>, line 1)

In [ ]:
#8th_step: We export the data to CSV format to facilitate further analysis.
filename = "essential_genes.csv"

with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Gene ID'])
    for gene_id in essential_genes:
        writer.writerow([gene_id])

#Also, we do this step for " non-essential genes" too.
with open("non_essential_genes.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Non-Essential Genes'])
    writer.writerows([[gene_id] for gene_id in non_essential_genes])


In [ ]:
#Our aim for the next step is to generate a plot that provides a comprehensive overview of the genes, making it easier to select the gene(s) for knockout.